# Working with OrderBook Data: NASDAQ ITCH

The order book is the primary source of market data, updated throughout the entire day in real time to reflect trading activity

The trading activity is reflected in numerous messages that conform to the electronic Financial Information eXchange (FIX) protocol for real-time exchange of securities transactions and market data

## Background

### The FIX Protocol 
The FIX Protocol is the standard for communication before and during trade execution between exchanges, banks, brokers, clearing firms, and other market participants.

Exchanges provide access to FIX messages as a real-time data feed that is parsed by algo traders to track market actiity

i.e. identifying the footprint of market participants and anticipate next moves.

### NASDAQ TotalView-ITCH Order Book data
FIX has a dominantly large market shares, exchanges also offer native protocols. The NASDAQ offers a **TotalView ITCH direct data-feed protocol** that allows subscribers to track individual orders for equity intstruments from placement to execution or cancellation.

It allows for the construction of the order book that keeps track of the list of active limit buy and sell orders for a security or financial instrument. The orderbook reveals the market depth throughout the day by listing the number of shares being bid or offered at each price point.

Market depth is a key indicator of liquidity and the potential price impact of sizable market orders.

The ITCH v5.0 spec. declares over 20 message types related to system events, stock characteristics, the placement and modification of limit orders, and trade execution.

## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import gzip
import shutil
from struct import unpack
from collections import namedtuple, Counter, defaultdict
from pathlib import Path
from urllib.request import urlretrieve
from urllib.parse import urljoin
from datetime import timedelta
from time import time

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [3]:
sns.set_style('whitegrid')

In [4]:
def format_time(t):
    """Return a formatted time string 'HH:MM:SS
    based on a numeric time() value"""
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>5.2f}'

## Get NASDAQ ITCH Data from FTP Server
Nasdaq offers samples of daily binary files for several months.

Code below illustrates how to parse a sample file of ITCH messages and reconstruct both the executed trades and the order book for any tick.

The following table shows the frequency of the most common message types for the sample file date March 29, 2018:

| Name                    | Offset  | Length  | Value      | Notes                                                                                |
|-------------------------|---------|---------|------------|--------------------------------------------------------------------------------------|
| Message Type            | 0       | 1       | S          | System Event Message                                                                 |
| Stock Locate            | 1       | 2       | Integer    | Always 0                                                                             |
| Tracking Number         | 3       | 2       | Integer    | Nasdaq internal tracking number                                                      |
| Timestamp               | 5       | 6       | Integer    | Nanoseconds since midnight                                                           |
| Order Reference Number  | 11      | 8       | Integer    | The unique reference number assigned to the new order at the time of receipt.        |
| Buy/Sell Indicator      | 19      | 1       | Alpha      | The type of order being added. B = Buy Order. S = Sell Order.                        |
| Shares                  | 20      | 4       | Integer    | The total number of shares associated with the order being added to the book.        |
| Stock                   | 24      | 8       | Alpha      | Stock symbol, right padded with spaces                                               |
| Price                   | 32      | 4       | Price (4)  | The display price of the new order. Refer to Data Types for field processing notes.  |
| Attribution             | 36      | 4       | Alpha      | Nasdaq Market participant identifier associated with the entered order               |

### Set Data Paths

Store the download in a 'data' subdirectory \
Convert results to hdf format

In [5]:
data_path = Path('data') # set to e.g. external harddrive
itch_store = str(data_path / 'itch.h5')
order_book_store = data_path / 'order_book.h5'

There are several sample files on the NASDAQ Server: https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/

In [6]:
HTTPS_URL = 'https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/'
SOURCE_FILE = '10302019.NASDAQ_ITCH50.gz'

### Download and Unzip Data

In [7]:
def may_be_download(url):
    """Download & unzip ITCH data if not yet available"""
    if not data_path.exists():
        print('Creating directory')
        data_path.mkdir()
    else:
        print('Directory exists')

    filename = data_path / url.split('/')[-1]
    if not filename.exists():
        print('Downloading...', url)
        urlretrieve(url, filename)
    else:
        print('File exists')

    unzipped = data_path / (filename.stem + '.bin')
    if not unzipped.exists():
        print('Unzipping to', unzipped)
        with gzip.open(str(filename), 'rb') as f_in:
            with open(unzipped, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        print('File already unpacked')
    return unzipped

In [8]:
file_name = may_be_download(urljoin(HTTPS_URL, SOURCE_FILE))
date = file_name.name.split('.')[0]

Creating directory
Downloading... https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/10302019.NASDAQ_ITCH50.gz
Unzipping to data/10302019.NASDAQ_ITCH50.bin


## ITCH Format Settings

## The `struct` module for binary data

ITCH tick data comes in binary format. Python provides the `struct` module to parse binary data using format strings that identify the message elements indicating length and type of the various components of the byte string

The following code blocks walk through the steps to parse trading messages and reconstruct the order book

In [9]:
event_codes = {'O': 'Start of Messages',
               'S': 'Start of System Hours',
               'Q': 'Start of Market Hours',
               'M': 'End of Market Hours',
               'E': 'End of System Hours',
               'C': 'End of Messages'}

In [10]:
encoding = {'primary_market_maker': {'Y': 1, 'N': 0},
            'printable'           : {'Y': 1, 'N': 0},
            'buy_sell_indicator'  : {'B': 1, 'S': -1},
            'cross_type'          : {'O': 0, 'C': 1, 'H': 2},
            'imbalance_direction' : {'B': 0, 'S': 1, 'N': 0, 'O': -1}}

In [11]:
formats = {
    ('integer', 2): 'H',  # int of length 2 => format string 'H'
    ('integer', 4): 'I',
    ('integer', 6): '6s',  # int of length 6 => parse as string, convert later
    ('integer', 8): 'Q',
    ('alpha',   1): 's',
    ('alpha',   2): '2s',
    ('alpha',   4): '4s',
    ('alpha',   8): '8s',
    ('price_4', 4): 'I',
    ('price_8', 8): 'Q',
}

### Create message specs for binary data parser